In [1]:
filePath = 'D:\DATA\SVMdata\sub4'
#運動実行のファイルパス
filepath1 = 'real'
#運動想起、両運動のファイルパス
filepath2 = 'image'
#test用データ
filepath3 = 'test'

#指定したチャネルを読み込む用のlist
readChanel = [9,12,17,20,31,44,46,49,54,57]
print(readChanel)

[9, 12, 17, 20, 31, 44, 46, 49, 54, 57]


In [2]:
#import library
import os
import numpy as np
import scipy.io
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pickle

In [3]:
# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65 = list()
R_data = list()  #  実運動のデータを格納
RI_data = list()  # 実運動＋想起運動のデータを格納
All_data = list()  # 全データを格納
#検証用
testo_data65 = list()
testR_data = list()  #  実運動のデータを格納
testRI_data = list()  # 実運動＋想起運動のデータを格納
testAll_data = list()  # 全データを格納

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    b = np.average(after_data)
    c = np.var(after_data)
    return after_data

In [4]:
file_name1 = list()  # すべての.matファイルの名前
file_name2 = list()
file_name3 = list()
for file in os.listdir(filePath+"\\"+filepath1):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name1 = sorted(file_name1)
        file_name1.append(file)
print('Filelist1')
print(file_name1)
for file in os.listdir(filePath+'\\'+filepath2):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name2 = sorted(file_name2)
        file_name2.append(file)
print('Filelist2')
print(file_name2)
for file in os.listdir(filePath+'\\'+filepath3):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name3 = sorted(file_name3)
        file_name3.append(file)
print('Filelist3')
print(file_name3)

Filelist1
['EEG_grasping_03-Mar-2020_matu_sub1-1_1day.mat', 'EEG_grasping_03-Mar-2020_matu_sub1-2_1day.mat', 'EEG_grasping_04-Mar-2020_matu_sub1-1_day2.mat', 'EEG_grasping_04-Mar-2020_matu_sub1-2_day2.mat', 'aEEG_grasping_03-Mar-2020_matu_sub1-3_1day.mat']
Filelist2
['EEG_grasping_03-Mar-2020_matu_sub2-1_1day.mat', 'EEG_grasping_03-Mar-2020_matu_sub2-2_1day.mat', 'EEG_grasping_03-Mar-2020_matu_sub3-1_1day.mat', 'EEG_grasping_03-Mar-2020_matu_sub3-2_1day.mat', 'EEG_grasping_04-Mar-2020_matu_sub2-1_day2.mat', 'EEG_grasping_04-Mar-2020_matu_sub2-2_day2.mat', 'EEG_grasping_04-Mar-2020_matu_sub2-3_day2.mat', 'EEG_grasping_04-Mar-2020_matu_sub3-1_day2.mat', 'EEG_grasping_04-Mar-2020_matu_sub3-2_day2.mat', 'EEG_grasping_04-Mar-2020_matu_sub3-3_day2.mat']
Filelist3
['EEG_grasping_03-Mar-2020_matu_sub2-3_1day.mat', 'EEG_grasping_03-Mar-2020_matu_sub3-3_1day.mat', 'aEEG_grasping_04-Mar-2020_matu_sub1-3_day2.mat']


In [5]:
# 変数など
data_directry = ''
file_num1 = len(file_name1)
file_num2 = len(file_name2)
file_num3 = len(file_name3)

trial_num = 30
all_trial = (file_num1+file_num2) * trial_num
all_d = 0
CH = 64
#運動実行を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+'\\'+filepath1+'\\'+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]

                #label = b[5][0, 0]
                label = 1

                o_data = O_Data(eeg, index, label)
                o_data65.append(o_data)
#運動想起を読み込む
for s in range(file_num2):
    Dictionary = scipy.io.loadmat(filePath+'\\'+filepath2+'\\'+file_name2[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]

                #label = b[5][0, 0]
                label = 0

                o_data = O_Data(eeg, index, label)
                o_data65.append(o_data)
testall_trial = file_num3*trial_num
#テストデータを読み込む
for s in range(file_num3):
    Dictionary = scipy.io.loadmat(filePath+'\\'+filepath3+'\\'+file_name3[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]

                label = 0
                if s >= file_num3-2:
                    label = 1

                testo_data = O_Data(eeg, index, label)
                testo_data65.append(testo_data)
print("data_loading")

data_loading


In [6]:
for n in range(all_trial):  # all_trial
    number = list()  # restのindex
    time = list()
    all_d = sum(len(v) for v in o_data65[n].index) - 1
    for i in range(all_d + 1):
        time.append(i)
    eeg = o_data65[n].eeg  # トライアルnのeeg
    index = o_data65[n].index  # トライアルnのindex
    label = o_data65[n].label  # トライアルnのlabel
    for i in range(all_d):
        if index[:, i] == 0:
            number.append(i)
    rest_time = len(number)  # rest時間のindex数

    # すべてのチャンネルで行う
    for j in range(len(readChanel)):
        onech_eeg = eeg[readChanel[j], :]  # 一つのチャンネルのeeg
        number = np.array(number)
        time = np.array(time)
        detrend = signal.detrend(onech_eeg[0:8000])  # ベースライン補正
        Standard = Standardization(detrend)# 正規化
        resample = signal.resample(Standard,4000)#ダウンサンプリング

        All_data.append(resample)
        
        if label == 1:
            #運動実行
            R_data.append(1)
        else:
            #運動想起
            R_data.append(0)

R_data = np.array(R_data)
All_data = np.array(All_data)

#テストデータ
for n in range(testall_trial):  # all_trial
    number = list()  # restのindex
    time = list()
    all_d = sum(len(v) for v in testo_data65[n].index) - 1
    for i in range(all_d + 1):
        time.append(i)
    eeg = testo_data65[n].eeg  # トライアルnのeeg
    index = testo_data65[n].index  # トライアルnのindex
    label = testo_data65[n].label  # トライアルnのlabel
    for i in range(all_d):
        if index[:, i] == 0:
            number.append(i)
    rest_time = len(number)  # rest時間のindex数

    # すべてのチャンネルで行う
    for j in range(len(readChanel)):
        onech_eeg = eeg[readChanel[j], :]  # 一つのチャンネルのeeg
        number = np.array(number)
        time = np.array(time)
        detrend = signal.detrend(onech_eeg[0:8000])  # ベースライン補正
        Standard = Standardization(detrend)#正規化
        resample = signal.resample(Standard,4000)#ダウンサンプリング

        testAll_data.append(resample)
        
        if label == 1:
            #運動実行
            testR_data.append(1)
        else:
            #運動想起
            testR_data.append(0)

testR_data = np.array(testR_data)
testAll_data = np.array(testAll_data)



print("trainning data")
print(np.shape(R_data))
print(np.shape(All_data))
print("test data")
print(np.shape(testR_data))
print(np.shape(testAll_data))

trainning data
(4500,)
(4500, 4000)
test data
(900,)
(900, 4000)


In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(All_data, R_data, test_size=0.25)



model =SVC(kernel='linear')
model.fit(X_train,Y_train)

SVC(C=1.0, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

0.6645925925925926
0.6728888888888889


In [8]:
#判別機を作る

SVC(C=1.0, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
  
fullmodel =SVC(kernel='linear')
fullmodel.fit(All_data,R_data)
#modelの保存
modelname = "modelSVCsub4.sav"
pickle.dump(fullmodel,open(modelname,'wb'))


In [ ]:
# 保存したモデルをロードする
#時系列データで検証
loaded_model = pickle.load(open(modelname, 'rb'))
result = loaded_model.score(testAll_data, testR_data)
print(result)

In [ ]:
params = {
    "C":np.arange(0.1,1,0.05),
    "kernel":["linear", "poly", "rbf", "sigmoid"],
    "gamma":np.arange(0.0001,0.1,0.05)
}
grid = GridSearchCV(fullmodel, params,scoring="accuracy", cv=5)
grid.fit(All_data,R_data)

print(grid.best_score_)
print(grid.best_params_)

pred = grid.predict(testAll_data)
print(classification_report(testR_data, pred))